https://ai.google.dev/gemini-api/docs/thinking

In [3]:
from kaggle_secrets import UserSecretsClient
from google import genai


user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Gemini")

client = genai.Client(
    api_key=secret_value_0,
    http_options={'api_version': 'v1alpha'},
)

In [21]:
# pip install --upgrade google-generativeai
# ?client.models.generate_content

In [23]:
delimiter = "####"
system_message = f"""
This is a bilateral trade game following VCG rules defined in VCG mechanism. \
In this game there are only two roles, buyer and seller. \
buyer and seller have their own evaluation for a good and that is their private information, \
in the meantime they also have declared evaluation. Both buyer and seller want to maximize their own utility. \
Remember, this is simotaneous game, buyer and seller give the declared evaluation at the same time and whether the trade would or wouldn't occur will be decided by that

Step 1:{delimiter} Think about what should be the declared evaluation

Step 2:{delimiter}: State your declared evaluation in the form of ##### Declared_Evaluation=Number #####
"""

In [25]:
prompt = f"""
You are a buyer, your private evaluation for the good is 5""".strip()

messages = [
    f"{system_message}",
    f"{prompt}",
]



In [28]:
response = client.models.generate_content(
    model='gemini-2.0-flash-thinking-exp',
    contents=messages,
)

# print(response.text)

buyer_thoughts = response.text

In [29]:
prompt = f"""
You are a seller, your private evaluation for the good is 3""".strip()

messages = [
    f"{system_message}",
    f"{prompt}",
]



In [30]:
response = client.models.generate_content(
    model='gemini-2.0-flash-thinking-exp',
    contents=messages,
)

print(response.text)

seller_thoughts = response.text

Step 1:#### Think about what should be the declared evaluation

In a VCG mechanism, truthful revelation of private information is a weakly dominant strategy. Let's analyze why this is the case for a seller in this bilateral trade game.

Let $v_s$ be the seller's private evaluation and $s$ be the declared evaluation. Let $v_b$ be the buyer's private evaluation and $b$ be the declared evaluation. Trade occurs if $b \ge s$.

**Scenario 1: Truthful Declaration (s = $v_s$ = 3)**

*   If the buyer declares $b \ge 3$, trade occurs. The seller receives a payment of $b$. The seller's utility is $b - v_s = b - 3$.
*   If the buyer declares $b < 3$, trade does not occur. The seller's utility is 0.

**Scenario 2: Over-declaration (s > $v_s$, e.g., s = 4)**

*   If the buyer declares $b \ge 4$, trade occurs. The seller receives a payment of $b$. The seller's utility is $b - v_s = b - 3$.
*   If the buyer declares $3 \le b < 4$, trade does not occur. The seller's utility is 0.  (If the seller had de

In [32]:
import re

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", buyer_thoughts)

if match:
    buyer_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {buyer_declared}")
else:
    print("No match found")

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", seller_thoughts)

if match:
    seller_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {seller_declared}")
else:
    print("No match found")

Extracted number: 5
Extracted number: 3


In [33]:
# import random

# def llm_buyer_decision(declared_valuation):
#     """Simulates a buyer's declared valuation based on private information using an LLM."""
#     # Example logic for declared valuation
#     # In practice, you would replace this with an API call to an LLM
#     declared_valuation = declared_valuation
#     return max(0, declared_valuation)  # Ensures valuation isn't negative

# def llm_seller_decision(declared_cost):
#     """Simulates a seller's declared cost based on private information using an LLM."""
#     # Example logic for declared valuation
#     # In practice, you would replace this with an API call to an LLM
#     declared_cost = declared_cost
#     return max(0, declared_cost)  # Ensures cost isn't negative

def vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation,declared_valuation,declared_cost, d=1):
    """Simulates bilateral trade with the VCG mechanism.

    Args:
        buyer_private_valuation: The buyer's true valuation.
        seller_private_valuation: The seller's true cost.
        d: Externality scaling factor.

    Returns:
        Tuple: (trade_occurs, price, buyer_utility, seller_utility)
    """
    # Buyer and seller use LLMs to decide their declared valuations
    buyer_declared_valuation = declared_valuation
    seller_declared_valuation = declared_cost

    # Seller's declared valuation with externality
    seller_declared_true_valuation = -d * seller_declared_valuation

    # Trade decision based on declared valuations
    if buyer_declared_valuation >= -seller_declared_true_valuation:
        trade_occurs = True
    else:
        trade_occurs = False

    if trade_occurs:
        # VCG price based on declared valuations
        price = -seller_declared_true_valuation

        # Buyer's utility (using private valuation)
        buyer_utility = buyer_private_valuation - price

        # Seller's transfer (using declared valuations)
        seller_transfer = d * buyer_declared_valuation - buyer_declared_valuation

        # Seller's utility (using private valuation)
        seller_utility = seller_transfer + (-d * seller_private_valuation)
    else:
        price = 0
        buyer_utility = 0
        seller_utility = -d * seller_private_valuation

    return (trade_occurs, price, buyer_utility, seller_utility, 
            buyer_declared_valuation, seller_declared_valuation)

# Example usage
buyer_private_valuation = 5  # Buyer knows their own valuation
seller_private_valuation = 3  # Seller knows their own cost

results = vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation, buyer_declared, seller_declared)

trade_occurs, price, buyer_utility, seller_utility, \
buyer_declared_valuation, seller_declared_valuation = results

print(f"Buyer private valuation: {buyer_private_valuation} \n")
print(f"Buyer declared valuation: {buyer_declared_valuation} \n")
print(f"Buyer thoughts: \n {buyer_thoughts} \n")
print(f"Seller private valuation (inherent cost): {seller_private_valuation} \n")
print(f"Seller declared valuation (inherent cost): {seller_declared_valuation} \n")
print(f"Buyer thoughts: \n {seller_thoughts} \n")

Buyer private valuation: 5 

Buyer declared valuation: 5 

Buyer thoughts: 
 #### Think about what should be the declared evaluation
In a VCG bilateral trade, trade occurs if the buyer's declared valuation (b) is greater than or equal to the seller's declared valuation (s). If trade occurs, the buyer pays the seller's declared valuation (s) and the seller receives the buyer's declared valuation (b).  For a buyer with a private valuation of 5, we need to consider the incentives to declare truthfully versus misreport.

Let $v_b = 5$ be the buyer's true valuation and $v_s$ be the seller's true valuation (which is unknown to the buyer, but the buyer needs to consider possible ranges of $s$). Let $b$ be the buyer's declared valuation and $s$ be the seller's declared valuation.

Trade happens if $b \ge s$.
If trade happens, buyer's utility is $U_b = v_b - s = 5 - s$.
If trade does not happen, buyer's utility is $U_b = 0$.

We consider if declaring the true valuation $b = 5$ is the optimal st

In [34]:
print(f"Trade occurs: {trade_occurs} \n")
if trade_occurs:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")
else:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")

Trade occurs: True 

Price: 3
Buyer utility: 2
Seller utility: -3
